In [ ]:
import scipy.io as sio
from torch.utils.data import DataLoader 
import torch
from torch import nn
import numpy as np
from torchsummary import summary
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load the training dataset from google drive
filename1 = '/content/drive/MyDrive/denoiser/deconvolver_sin1.mat'
batch_size = 64

data1 = sio.loadmat(filename1)

train1 = data1['train']
test1 = data1['test']

train_data = DataLoader(train1,batch_size=batch_size,shuffle=True,num_workers=2)
test_data = DataLoader(test1,batch_size=batch_size,shuffle=True,num_workers=2)

In [ ]:
# Define the CNN
class denoiser(nn.Module):
  def __init__(self):
    super(denoiser,self).__init__()

    self.C0 = nn.Conv1d(1,2,kernel_size=17,padding=8,bias=False)

    self.C1 = nn.Conv1d(2,1,kernel_size=17,padding=8,bias=False)

    self.C2 = nn.Conv1d(2,1,kernel_size=17,padding=8,bias=False)

    self.C3 = nn.Conv1d(2,2,kernel_size=37,padding=18,bias=False)
    
    self.C4 = nn.Conv1d(2,1,kernel_size=9,padding=4,bias=False)

  # Contrained training
  def forward(self,x):

    x_1 = F.relu(self.C0(x))

    # In layer 2, contrain that the filter (1,1) is the same as (2,2), and (1,2) is the same as (2,1)
    xp_2 = F.relu(self.C1(x_1))                         
    xn_2 = F.relu(F.conv1d(x_1,torch.flip(self.C1.weight,[1]),padding=8))
    x_2 = torch.cat((xp_2,xn_2),axis=1)

    # In layer 3, contrain that the filter (1,1) is the same as (2,2), and (1,2) is the same as (2,1)
    xp_3 = F.relu(self.C2(x_2))                    
    xn_3 = F.relu(F.conv1d(x_2,torch.flip(self.C2.weight,[1]),padding=8))
    x_3 = torch.cat((xp_3,xn_3),axis=1)

    x = F.relu(self.C3(x_3))
    x = self.C4(x)

    return x

In [ ]:
model = denoiser()
# Load the pre-set CNN as initialization
weight = sio.loadmat('/content/drive/MyDrive/denoiser/sin23_2.mat')
model._modules['C0']._parameters['weight'].data = torch.from_numpy(weight['conv1'])
model._modules['C3']._parameters['weight'].data = torch.from_numpy(weight['conv4'])
model._modules['C4']._parameters['weight'].data = torch.from_numpy(weight['conv5'])

# Set non-trainable layers
for name,value in model.named_parameters():
  if (name=='C0.weight')|(name=='C3.weight')|(name=='C4.weight'):
    value.requires_grad = False

In [ ]:
# Set training parameters
epochs = 700
device = torch.device("cuda:0")
model = model.to(device)
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr=1e-3)
#optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=300,gamma=0.1)
loss = nn.MSELoss().to(device)
summary(model, (1, 500))

In [ ]:
# Train the CNN
train_loss_epoch = []
test_loss_epoch = []
for epoch in range(epochs):
  train_losses = []
  test_losses = []
  for training in train_data:
    model.train()
    x_train = training[:,0,:].view(-1,1,500).type(torch.FloatTensor).to(device)
    y_train = training[:,2,:].view(-1,1,500).type(torch.FloatTensor).to(device)
    optimizer.zero_grad()
    y_pred = model(x_train)
    loss_train = loss(y_pred,y_train)
    loss_train.backward()
    optimizer.step()
    train_losses.append(loss_train.item())
  train_loss_epoch.append(np.mean(train_losses))
  scheduler.step()

  for testing in test_data:
    model.eval()
    x_test = testing[:,0,:].view(-1,1,500).type(torch.FloatTensor).to(device)
    y_test = testing[:,2,:].view(-1,1,500).type(torch.FloatTensor).to(device)
    y_pred = model(x_test)
    loss_test = loss(y_pred,y_test)
    test_losses.append(loss_test.item())
  test_loss_epoch.append(np.mean(test_losses))

  if (epoch%5)==0:
    print("Epoch: %d      train loss: %f      test loss: %f" %(epoch,np.mean(train_losses),np.mean(test_losses)))

  if ((epoch%100)==99)|(epoch==0):
    signal = y_test.view(-1,500).cpu().detach().numpy()
    denoised = y_pred.view(-1,500).cpu().detach().numpy()
    noisy = x_test.view(-1,500).cpu().detach().numpy()
    plt.figure(figsize=(10,10))
    plt.subplot(3,1,1)
    plt.plot(signal[0])
    plt.title('Pure Signal')
    plt.subplot(3,1,2)
    plt.plot(denoised[0])
    plt.title('Denoised Signal')
    plt.subplot(3,1,3)
    plt.plot(noisy[0])
    plt.title('Signal with Noise')


In [ ]:
# Save the trained CNN
weight_trained = dict()
weight_trained['conv1'] = model._modules['C0']._parameters['weight'].data.cpu().numpy()
weight_trained['conv2'] = model._modules['C1']._parameters['weight'].data.cpu().numpy()
weight_trained['conv3'] = model._modules['C2']._parameters['weight'].data.cpu().numpy()
weight_trained['conv4'] = model._modules['C3']._parameters['weight'].data.cpu().numpy()
weight_trained['conv5'] = model._modules['C4']._parameters['weight'].data.cpu().numpy()
sio.savemat('/content/drive/MyDrive/denoiser/deconv122_9.mat', mdict=weight_trained)